In [ ]:
%pip install transformers ipywidgets vllm nltk datasets huggingface_hub

In [2]:
import json
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import random
import json
import nltk
from datasets import load_dataset
from huggingface_hub import login
from generate_pii_dataset import generate_pii_dataset
from utils.pii_injection_utils import (
    get_pii_list,
    generate_data_transition,
    generate_text_transition,
)

ImportError: cannot import name 'get_full_repo_name' from 'huggingface_hub' (/root/miniconda3/envs/pii/lib/python3.10/site-packages/huggingface_hub/__init__.py)

## Load Model and Datasets

In [2]:
# Login to huggingface

login()

In [3]:
# Load the model and tokenizer
model_id = "meta-llama/Meta-Llama-3-8B-Instruct" 

model = LLM(model=model_id, gpu_memory_utilization=0.9, tensor_parallel_size=1)
tokenizer = AutoTokenizer.from_pretrained(model_id)

INFO 06-01 19:59:43 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B-Instruct)


/root/miniconda3/envs/pii/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-01 19:59:43 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
WARNING 06-01 19:59:43 utils.py:465] Using 'pin_memory=False' as WSL is detected. This may slow down the performance.


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 500: named symbol not found

In [4]:
# Generate or load PII samples
with open('grouped_pii_samples.json', 'r') as f:
    grouped_pii_samples = json.load(f)

grouped_pii_samples[0]

{'name': [],
 'email': ['ilpisug@zot.ci',
  'WAYNEvagjuv@ize.sl',
  'SKINNY@bibdivo.su',
  'Holt@ginsaufo.es',
  'Elvapattersonsahavi@ho.re'],
 'username': [],
 'id_number': ['6919978449503183',
  '019847530123598764',
  '13789',
  '78-47--2-2-5--32-96--5052836',
  '4797_4_350'],
 'phone_number': ['02 (320 537) 8935203', ' (823) 276 3 1564'],
 'street_address': [],
 'url': []}

In [5]:
# Load source texts
essay_dataset = load_dataset("qwedsacf/ivypanda-essays")
essay_dataset

DatasetDict({
    train: Dataset({
        features: ['TEXT', 'SOURCE', '__index_level_0__'],
        num_rows: 128293
    })
})

## Test PII Injection

In [18]:
# Set pii sample
pii_sample = grouped_pii_samples[0]
pii_sample

{'name': [],
 'email': ['ilpisug@zot.ci',
  'WAYNEvagjuv@ize.sl',
  'SKINNY@bibdivo.su',
  'Holt@ginsaufo.es',
  'Elvapattersonsahavi@ho.re'],
 'username': [],
 'id_number': ['6919978449503183',
  '019847530123598764',
  '13789',
  '78-47--2-2-5--32-96--5052836',
  '4797_4_350'],
 'phone_number': ['02 (320 537) 8935203', ' (823) 276 3 1564'],
 'street_address': [],
 'url': []}

In [19]:
# Set text to be injected with PII
text = essay_dataset['train'][0]['TEXT']
print(text)

12 Years a Slave: An Analysis of the Film Essay

The 2013 film 12 Years a Slave proved that slavery is a worldwide issue. Indeed, the film made $150 million outside the United States and $57 million in the U.S., with a production budget of $20 million (Sharf, 2020). The movie was based on the memoir Twelve Years a Slave by Solomon Northup (Ntim, 2020). It tells the story of a free African American man who was kidnapped and sold into slavery. Solomon spent twelve years away from his family, being traded from one master to another. Fortunately, the protagonist met a person who helped him deliver a message to his family and friends, who came and rescued him. This movie accurately illustrates discriminatory relationships between white slaveholders and black slaves that stemmed from the dysfunctional system in the country and prejudices in people’s mindsets at that time.

The two main ethnic groups presented in this film are White and African Americans, and the three social groups are afflu

In [29]:
# This function inserts PII into the text
def generate_pii_text(model, tokenizer, sampling_params, text, pii_map, split_by_sentence=True):
    splitted_text = nltk.sent_tokenize(text) if split_by_sentence else text.split(' ')
    pii_list = get_pii_list(pii_map)
    print(pii_list)

    for (pii, label) in pii_list:
        pii_insert_index = random.randint(0, len(splitted_text))
        first_text = ' '.join(splitted_text[:pii_insert_index])
        second_text = ' '.join(splitted_text[pii_insert_index:])

        transition = generate_data_transition(model, tokenizer, sampling_params, first_text, data=pii, data_type=label)
        transition_before = transition
        first_text = f"{first_text} {transition_before} {pii}"

        transition = generate_text_transition(model, tokenizer, sampling_params, first_text=first_text, second_text=second_text)
        transition_after = transition

        splitted_text.insert(pii_insert_index, f"{transition_before} {pii} {transition_after}")
    
    pii_text = ' '.join(splitted_text)

    return pii_text

In [30]:
terminators = [tokenizer.eos_token]

sampling_params = SamplingParams(
    temperature=0.6, 
    top_p=0.9, 
    max_tokens=2058, 
    skip_special_tokens=True,
    stop=terminators
)


print(generate_pii_text(model, tokenizer, sampling_params, text, pii_sample, split_by_sentence=False))

[('Elvapattersonsahavi@ho.re', 'email'), ('78-47--2-2-5--32-96--5052836', 'id_number'), ('SKINNY@bibdivo.su', 'email'), (' (823) 276 3 1564', 'phone_number'), ('ilpisug@zot.ci', 'email'), ('6919978449503183', 'id_number'), ('13789', 'id_number'), ('WAYNEvagjuv@ize.sl', 'email'), ('Holt@ginsaufo.es', 'email'), ('02 (320 537) 8935203', 'phone_number'), ('019847530123598764', 'id_number'), ('4797_4_350', 'id_number')]


TypeError: generate_data_transition() got multiple values for argument 'data'

## Test Generate Dataset

In [6]:
output_dataset_name_path = "pii_dataset"
max_dataset_size = 10

terminators = [tokenizer.eos_token]

sampling_params = SamplingParams(
    temperature=0.6, 
    top_p=0.9, 
    max_tokens=2058, 
    skip_special_tokens=True,
    stop=terminators
)

# Generate PII Dataset
pii_dataset = generate_pii_dataset(model, tokenizer, sampling_params, essay_dataset['train']['TEXT'], grouped_pii_samples, output_dataset_name_path, max_dataset_size = max_dataset_size)

print(f"\nFinished Generating PII Dataset with {len(pii_dataset)} samples\n")


Processed prompts: 100%|██████████| 1/1 [00:12<00:00, 12.37s/it]


In [12]:
pii_dataset[0]

{'source_text': '12 Years a Slave: An Analysis of the Film Essay\n\nThe 2013 film 12 Years a Slave proved that slavery is a worldwide issue. Indeed, the film made $150 million outside the United States and $57 million in the U.S., with a production budget of $20 million (Sharf, 2020). The movie was based on the memoir Twelve Years a Slave by Solomon Northup (Ntim, 2020). It tells the story of a free African American man who was kidnapped and sold into slavery. Solomon spent twelve years away from his family, being traded from one master to another. Fortunately, the protagonist met a person who helped him deliver a message to his family and friends, who came and rescued him. This movie accurately illustrates discriminatory relationships between white slaveholders and black slaves that stemmed from the dysfunctional system in the country and prejudices in people’s mindsets at that time. The two main ethnic groups presented in this film are White and African Americans, and the three socia

In [11]:
# Print the first 5 samples
print("First 5 samples:")
for i in range(5):
    if i >= len(pii_dataset):
        break
    print(f"Sample {i+1}:")
    print(f"\nSource Text: {pii_dataset[i]['source_text']}")
    print(f"\nPII Text: {pii_dataset[i]['pii_text']}")
    print(f"\nPII Data: {pii_dataset[i]['pii_data']}")
    print(f"\nPII Labels: {pii_dataset[i]['pii_labels']}")
    print("\n")

In [9]:
# Print the first 5 samples
print("First 5 samples:")
for i in range(5):
    if i >= len(pii_dataset):
        break
    print(f"Sample {i+1}:")
    print(f"\nSource Text: {pii_dataset[i]['source_text']}")
    print(f"\nPII Text: {pii_dataset[i]['pii_text']}")
    print(f"\nPII Data: {pii_dataset[i]['pii_data']}")
    print(f"\nPII Labels: {pii_dataset[i]['pii_labels']}")
    print("\n")